In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
file = '/Users/Lindiway/MACHINE_LEARNING_2025/FEB-ML-25-Multi-Partner/RecommenderEngine/books.csv'
df = df = pd.read_csv(file, on_bad_lines='skip')
df.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.


In [ ]:
len_bookid = len(df['bookID'].unique())
len_bookid

11123

In [6]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

## Use a Weighted Rating
<font color = "White"> Create a function named popularity Recommender and use it to recommend books based on popularity. <br>
Using a weighted rank similar to that used in the IMDB rating 


In [11]:
def popularityRecommender(df):
    
    #Define the minimum vote count
    minimum_rating_count = 0.75* df['ratings_count'].max()
    
    #Define C – the mean rating
    mean_rating = df['average_rating'].mean()

    df['weighted_rating'] = (((df['ratings_count']/(df['ratings_count']+minimum_rating_count)) * df['average_rating']) +
                             ((minimum_rating_count/(df['ratings_count']+minimum_rating_count))*mean_rating))

    recommendations = df.sort_values(by = 'weighted_rating',ascending = False).head(5)
    
    return(recommendations) 


In [12]:
top5 = popularityRecommender(df)
top5[["title",'ratings_count','average_rating','weighted_rating',]].head(5)

,title,ratings_count,average_rating,weighted_rating
3,Harry Potter and the Prisoner of Azkaban (Harr...,2339585,4.56,4.187089
0,Harry Potter and the Half-Blood Prince (Harry ...,2095690,4.57,4.174464
1,Harry Potter and the Order of the Phoenix (Har...,2153167,4.49,4.147771
4415,Harry Potter and the Chamber of Secrets (Harry...,2293963,4.42,4.128198
23,The Fellowship of the Ring (The Lord of the Ri...,2128944,4.36,4.096661


In [13]:
#The top 5 recommended books based on popularity are:
top5Names = top5["title"].head(5).values
print(f'The top 5 recommended books based on popularity are:\n {top5Names}')

The top 5 recommended books based on popularity are:
 ['Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)'
 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)'
 'Harry Potter and the Order of the Phoenix (Harry Potter  #5)'
 'Harry Potter and the Chamber of Secrets (Harry Potter  #2)'
 'The Fellowship of the Ring (The Lord of the Rings  #1)']


### <font color = "White"> Content Based Recommender
<font color = "White"> Create a function named Content Based Recommender with TF-IDF and use it to recommend books based on content.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


df['content'] = df['title'] + ' ' + df['authors']  # Build the content column

cbr = TfidfVectorizer(stop_words='english')  # Create a tfidf matrix
tfidf_matrix = cbr.fit_transform(df['content'])
distance_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix) #Compute cosine similarity
indices = pd.Series(df.index, index=df['title']).drop_duplicates() # Build a reverse index (title to index)

# 5. Content-based recommender function
def ContentBasedRecommender(title, indices, distance_matrix, top_n=5):
    if title not in indices:
        print(f"'{title}' not found in the dataset.")
        return []

    idx = indices[title]
    distance = list(enumerate(distance_matrix[idx]))
    distance = sorted(distance, key=lambda x: x[1], reverse=True)
    distance = distance[1:top_n+1]  # skip the first (the book itself)

    book_indices = [i[0] for i in distance]
    return df['title'].iloc[book_indices]


In [25]:

book_title = "Harry Potter and the Order of the Phoenix (Harry Potter  #5)"
recommendations = ContentBasedRecommender(book_title, indices, distance_matrix, top_n=5)
print(recommendations)

4415    Harry Potter and the Chamber of Secrets (Harry...
8873    Harry Potter and the Sorcerer's Stone (Harry P...
0       Harry Potter and the Half-Blood Prince (Harry ...
3       Harry Potter and the Prisoner of Azkaban (Harr...
4       Harry Potter Boxed Set  Books 1-5 (Harry Potte...
Name: title, dtype: object
